In [ ]:
!pip install cirq

     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 1.6 MB 10.1 MB/s 
     |████████████████████████████████| 328 kB 30.6 MB/s 
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 437 kB 47.7 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 145 kB 55.1 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 220 kB 67.2 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 10.1 MB 31.3 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 229 kB 55.4 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 15.7 MB 106 kB/s 
     |███████████████████████

In [ ]:
import cirq

In [ ]:
print(cirq.google.Sycamore)

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                    (1, 4)───(1, 5)───(1, 6)───(1, 7)
                                    │        │        │        │
                                    │        │        │        │
                           (2, 3)───(2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                           │        │        │        │        │        │
                           │        │        │        │        │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │        │        │        │        │
                  │        │        │        │        │        │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)───(4, 6)───(4, 7)───(4, 8)───(4, 9)
         │        │        │        │        │        │   

Experimenting and building simple functionalities : And and 1-bit addition

**And gate**

In [ ]:
a = cirq.NamedQubit('a')
b = cirq.NamedQubit('b')
c = cirq.NamedQubit('c')

In [ ]:
circuit = cirq.Circuit(
    cirq.H(a),
    cirq.H(b),
    cirq.TOFFOLI(a,b,c),
    cirq.measure(a,b,c)
)

In [ ]:
print(circuit)

a: ───H───@───M───
          │   │
b: ───H───@───M───
          │   │
c: ───────X───M───


In [ ]:
sim = cirq.Simulator()

In [ ]:
result = sim.run(circuit,repetitions=10)
print(result)

a,b,c=0100101101, 0001111110, 0000101100


**1-bit Addition**

In [ ]:
a = cirq.NamedQubit('a')
b = cirq.NamedQubit('b')
c = cirq.NamedQubit('c')

In [ ]:
circuit_add = cirq.Circuit(
    cirq.H(a),
    cirq.H(b),
    cirq.measure(a,b),
    cirq.TOFFOLI(a,b,c),
    cirq.CNOT(a,b),
    cirq.measure(c,b)
)
print(circuit_add)

a: ───H───M───@───@───────
          │   │   │
b: ───H───M───@───X───M───
              │       │
c: ───────────X───────M───


In [ ]:
sim = cirq.Simulator()

In [ ]:
result = sim.run(circuit_add,repetitions=10)
print(result)

a,b=1111000110, 1111000001
c,b=1111000000, 0000000111


**Quantum Teleportation**

In [ ]:
import random
import numpy as np
import cirq

In [ ]:

#initialising randoms for creating random message
ranx=random.random()
rany=random.random()
circuit = cirq.Circuit()
#assigning qubits
msg, alice, bob = cirq.LineQubit.range(3)

circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])

circuit.append([cirq.X(msg) ** ranx, cirq.Y(msg) ** rany])

circuit.append([cirq.CNOT(msg, alice), cirq.H(msg)])
circuit.append(cirq.measure(msg, alice))

circuit.append([cirq.CNOT(alice, bob), cirq.CZ(msg, bob)])

In [ ]:
print("Circuit:")
print(circuit)

Circuit:
0: ───X^0.89───Y^(1/7)───@───H───M───────@───
                         │       │       │
1: ───H────────@─────────X───────M───@───┼───
               │                     │   │
2: ────────────X─────────────────────X───@───


Last cnot gate and CZ gate in real world are for communicationg the measured message at alices side

In [ ]:
sim = cirq.Simulator()
#TO see the message before hand
q0 = cirq.LineQubit(0)
message = sim.simulate(cirq.Circuit([cirq.X(q0) ** ranx, cirq.Y(q0) ** rany]))

print("\nBloch sphere of randomly made Message:")

expected = cirq.bloch_vector_from_state_vector(message.final_state_vector, 0)
print(
    "x: ",
    np.around(expected[0], 4),
    "y: ",
    np.around(expected[1], 4),
    "z: ",
    np.around(expected[2], 4),
)


final_results = sim.simulate(circuit)

print("\nBloch Sphere of BoB qubit at Final State:")

teleported = cirq.bloch_vector_from_state_vector(final_results.final_state_vector, 2)
print(
    "x: ",
    np.around(teleported[0], 4),
    "y: ",
    np.around(teleported[1], 4),
    "z: ",
    np.around(teleported[2], 4),
)




Bloch sphere of randomly made Message:
x:  -0.4069 y:  -0.3393 z:  -0.8481

Bloch Sphere of BoB qubit at Final State:
x:  -0.4069 y:  -0.3393 z:  -0.8481


The Quantum information has teleported to BoB perfectly